<a href="https://colab.research.google.com/github/MarshallW11/MLProject/blob/main/MLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
features = pd.read_csv('/content/drive/MyDrive/project_files/Features data set.csv')
sales = pd.read_csv('/content/drive/MyDrive/project_files/sales data-set.csv')
stores = pd.read_csv('/content/drive/MyDrive/project_files/stores data-set.csv')

In [ ]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [ ]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [ ]:
sales.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [ ]:
features = features.drop(['IsHoliday'], axis=1)

combined = sales.merge(features, on=['Store', 'Date'])
combined = combined.merge(stores, on=['Store'])

combined.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,05/02/2010,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,1,12/02/2010,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315
2,1,1,19/02/2010,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315
3,1,1,26/02/2010,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315
4,1,1,05/03/2010,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315


In [ ]:
sales_combinedDept = sales.groupby(['Store', 'Date'], as_index=False).agg({'Weekly_Sales': 'sum'})

sales_combinedDept['Date'] = pd.to_datetime(sales_combinedDept['Date'], dayfirst=True)
sales_combinedDept = sales_combinedDept.sort_values(by = 'Date')
sales_combinedDept.head()

,Store,Date,Weekly_Sales
5310,38,2010-02-05,358496.14
1735,13,2010-02-05,1967220.53
5882,42,2010-02-05,543384.01
3451,25,2010-02-05,677231.63
4166,30,2010-02-05,465108.52


In [ ]:
output_dir = '/content/drive/MyDrive/project_files/SalesOverTime'
os.makedirs(output_dir, exist_ok=True)


for store_id in sales_combinedDept['Store'].unique():
    store_data = sales_combinedDept[sales_combinedDept['Store'] == store_id]

    plt.figure(figsize=(10, 6))
    plt.plot(store_data['Date'], store_data['Weekly_Sales'], label=f'Store {store_id}', color='blue')
    plt.xlabel('Date')
    plt.ylabel('Weekly Sales')
    plt.title(f'Weekly Sales for Store {store_id}')
    plt.legend()

    filename = os.path.join(output_dir, f'Store_{store_id}_sales.png')
    plt.savefig(filename)
    plt.close()

print(f"Plots saved in folder: {output_dir}")

Plots saved in folder: /content/drive/MyDrive/project_files/SalesOverTime


In [ ]:
output_dir = 's/content/drive/MyDrive/project_files/StoreSalesPlots'
os.makedirs(output_dir, exist_ok=True)

x = 5

store_ids = sales_combinedDept['Store'].unique()
store_chunks = np.array_split(store_ids, len(store_ids) // x + (len(store_ids) % x > 0))

for i, store_chunk in enumerate(store_chunks):
    plt.figure(figsize=(10, 6))

    for store_id in store_chunk:
        store_data = sales_combinedDept[sales_combinedDept['Store'] == store_id]

        plt.plot(store_data['Date'], store_data['Weekly_Sales'], label=f'Store {store_id}')

    plt.xlabel('Date')
    plt.ylabel('Weekly Sales')
    plt.title(f'Weekly Sales for Stores {store_chunk.tolist()}')
    plt.legend()

    filename = os.path.join(output_dir, f'Stores_{i + 1}_batch_sales.png')
    plt.savefig(filename)
    plt.close()

print(f"Plots saved in folder: {output_dir}")

Plots saved in folder: s/content/drive/MyDrive/project_files/StoreSalesPlots


In [ ]:
combined['Date'] = pd.to_datetime(combined['Date'], dayfirst=True)
combined['Year'] = combined['Date'].dt.year
combined['Month'] = combined['Date'].dt.month
combined['WeekOfYear'] = combined['Date'].dt.isocalendar().week
combined['DayOfWeek'] = combined['Date'].dt.dayofweek
combined.sort_values(by='Date', inplace=True)
combined.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,WeeekOfYear,DayOfWeek
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315,2010,2,5,4
277665,29,5,2010-02-05,15552.08,False,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,B,93638,2010,2,5,4
277808,29,6,2010-02-05,3200.22,False,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,B,93638,2010,2,5,4
277951,29,7,2010-02-05,10820.05,False,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,B,93638,2010,2,5,4
278094,29,8,2010-02-05,20055.64,False,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,B,93638,2010,2,5,4


In [ ]:
#Not sure if these fields are needed, we can play around with these calculations, see if they help, but I think the ones below, with the weighted values, are what we should focuse on

# store_sales = combined.groupby('Store').agg({
   # 'Weekly_Sales': ['mean', 'max', 'std']
#}).reset_index()
#store_sales.columns = ['Store', 'Avg_Weekly_Sales', 'Max_Weekly_Sales', 'Sales_Volatility']
#combined = combined.merge(store_sales, on='Store')
#combined.head()

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)

# Initialize a Holiday_Weight column with a default value of 1
combined['Holiday_Weight'] = 1

# Define holiday weeks and set weight to 5 for each
# Assuming Super Bowl is the first Sunday in February, adjust as necessary
super_bowl_week = (combined['Date'].dt.month == 2) & (combined['Date'].dt.isocalendar().week <= 6) & (combined['Date'].dt.dayofweek == 6)
labor_day_week = (combined['Date'].dt.month == 9) & (combined['Date'].dt.isocalendar().week == 36)
thanksgiving_week = (combined['Date'].dt.month == 11) & (combined['Date'].dt.isocalendar().week == 47)
christmas_period = (combined['Date'].dt.month == 12) & (combined['Date'].dt.isocalendar().week <= 24)

# Set the holiday weights for each holiday period
combined.loc[super_bowl_week, 'Holiday_Weight'] = 5
combined.loc[labor_day_week, 'Holiday_Weight'] = 5
combined.loc[thanksgiving_week, 'Holiday_Weight'] = 5
combined.loc[christmas_period, 'Holiday_Weight'] = 5

# Weighted Weekly Sales
combined['Weighted_Weekly_Sales'] = combined['Weekly_Sales'] * combined['Holiday_Weight']

# Calculates the weighted average sales by store, taking holiday weights into account
weighted_sales = combined.groupby('Store', group_keys=False).apply(
    lambda x: np.average(x['Weekly_Sales'], weights=x['Holiday_Weight']),
    include_groups=False #Eclude grouping columns from the operation
).reset_index(name='Weighted_Avg_Weekly_Sales')
weighted_sales.columns = ['Store', 'Weighted_Avg_Weekly_Sales']

# Merge back to add weighted sales metric to the original DataFrame if needed
# Check if 'Weighted_Avg_Weekly_Sales' already exists, and drop it if it does
if 'Weighted_Avg_Weekly_Sales' in combined.columns:
    combined = combined.drop(columns=['Weighted_Avg_Weekly_Sales'])  # Drop the existing column
#If Weighted_Avg_Weekly_Sales_original' already exists from a previous merge, drop it
if 'Weighted_Avg_Weekly_Sales_original' in combined.columns:
    combined = combined.drop(columns=['Weighted_Avg_Weekly_Sales_original'])

# Merge back to add weighted sales metric to the original DataFrame if needed
combined = combined.merge(weighted_sales, on='Store', how='left')
combined.head()


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,WeeekOfYear,DayOfWeek,Holiday_Weight,Weighted_Weekly_Sales,Weighted_Avg_Weekly_Sales
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315,2010,2,5,4,1,24924.50,21997.723655
1,29,5,2010-02-05,15552.08,False,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,B,93638,2010,2,5,4,1,15552.08,8412.383227
2,29,6,2010-02-05,3200.22,False,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,B,93638,2010,2,5,4,1,3200.22,8412.383227
3,29,7,2010-02-05,10820.05,False,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,B,93638,2010,2,5,4,1,10820.05,8412.383227
4,29,8,2010-02-05,20055.64,False,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,B,93638,2010,2,5,4,1,20055.64,8412.383227


In [ ]:

combined[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']] = combined[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].replace("", np.nan)

combined[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']] = combined[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].fillna(0)

missing_values_check = combined[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].isnull().sum()

print("Missing values after filling:")
print(missing_values_check)

Missing values after filling:
MarkDown1    0
MarkDown2    0
MarkDown3    0
MarkDown4    0
MarkDown5    0
dtype: int64


In [ ]:
# Select only numeric columns and exclude Weekly_Sales from the correlation analysis
numeric_df = combined.select_dtypes(include=[np.number]).drop(columns=['Weekly_Sales'])

# Compute correlation matrix
correlation = numeric_df.corrwith(combined['Weekly_Sales']).sort_values(ascending=False)

print("Correlation of each feature with Weekly_Sales:")
print(correlation)

Correlation of each feature with Weekly_Sales:
Weighted_Weekly_Sales        0.739697
Weighted_Avg_Weekly_Sales    0.305254
Size                         0.243828
Dept                         0.148032
MarkDown5                    0.050465
MarkDown1                    0.047172
MarkDown3                    0.038562
MarkDown4                    0.037467
Month                        0.028409
WeeekOfYear                  0.027673
MarkDown2                    0.020716
Holiday_Weight               0.020586
Fuel_Price                  -0.000120
Temperature                 -0.002312
Year                        -0.010111
CPI                         -0.020921
Unemployment                -0.025864
Store                       -0.085195
DayOfWeek                         NaN
dtype: float64


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA()
scaler = StandardScaler()

#making a copy to preserve the original unscaled df
combined_scaled = combined.copy()

#scaling each of the numerical columns
combined_scaled['Weekly_Sales'] = scaler.fit_transform(combined_scaled[['Weekly_Sales']])
combined_scaled['Temperature'] = scaler.fit_transform(combined_scaled[['Temperature']])
combined_scaled['Fuel_Price'] = scaler.fit_transform(combined_scaled[['Fuel_Price']])
combined_scaled['MarkDown1'] = scaler.fit_transform(combined_scaled[['MarkDown1']])
combined_scaled['MarkDown2'] = scaler.fit_transform(combined_scaled[['MarkDown2']])
combined_scaled['MarkDown3'] = scaler.fit_transform(combined_scaled[['MarkDown3']])
combined_scaled['MarkDown4'] = scaler.fit_transform(combined_scaled[['MarkDown4']])
combined_scaled['MarkDown5'] = scaler.fit_transform(combined_scaled[['MarkDown5']])
combined_scaled['CPI'] = scaler.fit_transform(combined_scaled[['CPI']])
combined_scaled['Unemployment'] = scaler.fit_transform(combined_scaled[['Unemployment']])
combined_scaled['Size'] = scaler.fit_transform(combined_scaled[['Size']])
combined_scaled['Weighted_Weekly_Sales'] = scaler.fit_transform(combined_scaled[['Weighted_Weekly_Sales']])
combined_scaled['Weighted_Avg_Weekly_Sales'] = scaler.fit_transform(combined_scaled[['Weighted_Avg_Weekly_Sales']])
combined_scaled.head()

#performing PCA on the numerical features. Which features we select will likely need tweaking
pca.fit(combined_scaled[['Weekly_Sales', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size', 'Weighted_Weekly_Sales', 'Weighted_Avg_Weekly_Sales']])
print(pca.explained_variance_ratio_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder


# Define features and target variable
X = combined.drop(columns=['Weekly_Sales', 'Date'])
label_encoder = LabelEncoder()


X['Type'] = label_encoder.fit_transform(X['Type'])
y = combined['Weekly_Sales']

# Initialize and fit Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# Calculate feature importances
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances from Random Forest:")
print(feature_importances)

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler



In [1]:
!git add .
!git commit -m "Testing upload to github"
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
